In [223]:
from selenium.webdriver import Chrome
import pymongo
import time
import pandas as pd
from bs4 import BeautifulSoup
import json

In [49]:
browser = Chrome()

In [50]:
mc = pymongo.MongoClient()
db = mc['avalanche']
forecast_coll = db['forecasts']

In [51]:
urls_df = pd.read_json('url_scrape.json')

In [52]:
display(urls_df.shape)
urls_df.head()

(346, 1)

,urls
0,https://www.nwac.us/avalanche-forecast/avalanc...
1,https://www.nwac.us/avalanche-forecast/avalanc...
10,https://www.nwac.us/avalanche-forecast/avalanc...
100,https://www.nwac.us/avalanche-forecast/avalanc...
101,https://www.nwac.us/avalanche-forecast/avalanc...


In [12]:
urls =list(urls_df['urls'])

'https://www.nwac.us/avalanche-forecast/avalanche-region-forecast/5608/cascade-west-north-baker/'

In [16]:
url = ('https://www.nwac.us/avalanche-forecast/avalanche-region-forecast/5176/'
        'cascade-west-stevens-pass/'
      )

In [14]:
def scrape_url(url, browser=browser, delay=5):
    """Returns the HTML source from a URL."""
    browser.get(url)
    time.sleep(delay)
    html = browser.page_source
    return html

In [15]:
def collect_page(url, browser=browser, coll=forecast_coll, delay=5):
    """Scrapes and saves the source of a web page."""
    docs = list(coll.find({'url': url}))
    if len(docs) == 0:
        html = scrape_url(browser=browser,
                          url=url,
                          delay=delay)
        coll.insert_one({
            'url': url,
            'html': html,
        })
    doc = coll.find_one({'url' : url})
    return doc
    
    

In [17]:
collect_page(url)

{'_id': ObjectId('5d113eff9d1587f4b57a6132'),
 'url': 'https://www.nwac.us/avalanche-forecast/avalanche-region-forecast/5176/cascade-west-stevens-pass/',
 'html': '<html class=" overthrow-enabled"><head>\n        <meta http-equiv="content-type" content="text/html; charset=utf-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\n        <meta name="description" content="Provides mountain weather and avalanche information for the Northwest US, including the Washington Cascades, Olympic Mountains and the northern Oregon Cascades.">\n<!--        <link rel="image_src" href="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg"> -->\n        <link rel="image_src" href="//www.nwac.us/static/common/images/nwac-logo-square-200.jpg">\n\n\n        <meta property="og:type" content="website">\n<!--         <meta property="og:image" content="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg" /> -->\n        <meta property="og:image" conten

In [18]:
for url in urls:
    collect_page(url)

In [53]:
db.list_collection_names()

['forecasts']

In [54]:
forecast_coll.count_documents({})

347

In [55]:
print(db.forecasts.find_one())

{'_id': ObjectId('5d113eff9d1587f4b57a6132'), 'url': 'https://www.nwac.us/avalanche-forecast/avalanche-region-forecast/5176/cascade-west-stevens-pass/', 'html': '<html class=" overthrow-enabled"><head>\n        <meta http-equiv="content-type" content="text/html; charset=utf-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\n        <meta name="description" content="Provides mountain weather and avalanche information for the Northwest US, including the Washington Cascades, Olympic Mountains and the northern Oregon Cascades.">\n<!--        <link rel="image_src" href="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg"> -->\n        <link rel="image_src" href="//www.nwac.us/static/common/images/nwac-logo-square-200.jpg">\n\n\n        <meta property="og:type" content="website">\n<!--         <meta property="og:image" content="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg" /> -->\n        <meta property="og:image" content=

In [56]:
count = 0
for page in db.forecasts.find():
    count += 1
print(count)

347


how to find the stevens pass ones? 

https://stackoverflow.com/questions/10242501/how-to-find-a-substring-in-a-field-in-mongodb

db.test.find({A: {$regex: 'Star Wars'}})


In [57]:
coll = forecast_coll

In [58]:
query= {'url': 'https://www.nwac.us/avalanche-forecast/'
           'avalanche-region-forecast/5176/cascade-west-stevens-pass/'}

In [59]:
first_cur = coll.find(query)

In [60]:
coll.count_documents(query)

1

In [40]:
next(first_cur)

{'_id': ObjectId('5d113eff9d1587f4b57a6132'),
 'url': 'https://www.nwac.us/avalanche-forecast/avalanche-region-forecast/5176/cascade-west-stevens-pass/',
 'html': '<html class=" overthrow-enabled"><head>\n        <meta http-equiv="content-type" content="text/html; charset=utf-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\n        <meta name="description" content="Provides mountain weather and avalanche information for the Northwest US, including the Washington Cascades, Olympic Mountains and the northern Oregon Cascades.">\n<!--        <link rel="image_src" href="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg"> -->\n        <link rel="image_src" href="//www.nwac.us/static/common/images/nwac-logo-square-200.jpg">\n\n\n        <meta property="og:type" content="website">\n<!--         <meta property="og:image" content="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg" /> -->\n        <meta property="og:image" conten

I can find the first one, now to find all that are stevens pass


In [61]:
cur = coll.find({})
stevens = [forecasts for forecasts in cur if 'stevens' in forecasts['url']]

In [62]:
len(stevens)

36

In [164]:
type(stevens)

list

In [67]:
stevens[0]['html']

'<html class=" overthrow-enabled"><head>\n        <meta http-equiv="content-type" content="text/html; charset=utf-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\n        <meta name="description" content="Provides mountain weather and avalanche information for the Northwest US, including the Washington Cascades, Olympic Mountains and the northern Oregon Cascades.">\n<!--        <link rel="image_src" href="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg"> -->\n        <link rel="image_src" href="//www.nwac.us/static/common/images/nwac-logo-square-200.jpg">\n\n\n        <meta property="og:type" content="website">\n<!--         <meta property="og:image" content="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg" /> -->\n        <meta property="og:image" content="//www.nwac.us/static/common/images/nwac-logo-square-200.jpg">\n        <meta property="og:image:type" content="image/jpeg">\n<!--         <meta property="og:url

In [84]:
soup = BeautifulSoup(stevens[0]['html'],'lxml')

In [85]:
print(soup.prettify())

<html class=" overthrow-enabled">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Provides mountain weather and avalanche information for the Northwest US, including the Washington Cascades, Olympic Mountains and the northern Oregon Cascades." name="description"/>
  <!--        <link rel="image_src" href="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg"> -->
  <link href="//www.nwac.us/static/common/images/nwac-logo-square-200.jpg" rel="image_src"/>
  <meta content="website" property="og:type"/>
  <!--         <meta property="og:image" content="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg" /> -->
  <meta content="//www.nwac.us/static/common/images/nwac-logo-square-200.jpg" property="og:image"/>
  <meta content="image/jpeg" property="og:image:type"/>
  <!--         <meta property="og:url" content="http://www.nwac.us/avalanche-fore

In [114]:
danger_above_tag = '#treeline-above > div.span4.elev-day1-column > div.danger-description > h4'
danger_near_tag ='#treeline-near > div.span4.elev-day1-column > div.danger-description > h4'
danger_below_tag = '#treeline-below > div.span4.elev-day1-column > div.danger-description > h4'
area_tag = '#main-content > h2'
date = '#elevation-levels-header > div.span4.desktop.elevation-day-name > p'

In [156]:
danger_above_tag = '#treeline-above > div.span4.elev-day1-column > div.danger-description > h4'
danger_near_tag ='#treeline-near > div.span4.elev-day1-column > div.danger-description > h4'
danger_below_tag = '#treeline-below > div.span4.elev-day1-column > div.danger-description > h4'
area_tag = '#main-content > h2'
date_tag = '#elevation-levels-header > div.span4.desktop.elevation-day-name > p'

In [230]:
def create_danger_info(page):
    '''Collects the relevant info from the webpage html'''
    
    
    danger_above_tag = '#treeline-above > div.span4.elev-day1-column > div.danger-description > h4'
    danger_near_tag ='#treeline-near > div.span4.elev-day1-column > div.danger-description > h4'
    danger_below_tag = '#treeline-below > div.span4.elev-day1-column > div.danger-description > h4'
    area_tag = '#main-content > h2'
    date_tag = '#elevation-levels-header > div.span4.desktop.elevation-day-name > p'
    soup = BeautifulSoup(page['html'],'lxml')
   
    area = soup.select_one(area_tag).text
    date = soup.select_one(date_tag).text
    danger_above = soup.select_one(danger_above_tag).text
    danger_near = soup.select_one(danger_near_tag).text
    danger_below = soup.select_one(danger_below_tag).text
    return {'date': date,
               'area': area,
               'danger_above_treeline': danger_above,
               'danger_near_treeline': danger_near,
               'danger_below_treeline': danger_below,
           }


In [231]:
avalanche_forcasts = [create_danger_info(page) for page in stevens]
avalanches = pd.DataFrame(avalanche_forcasts)


In [242]:
avalanches = avalanches.drop_duplicates()

In [243]:
avalanches.to_json('stevens_avalanches.json')

In [244]:
df = pd.read_json('stevens_avalanches.json')


In [246]:
df.shape

(33, 5)

In [150]:
for page in stevens:
    create_danger_list(page)
    
    

In [210]:
danger_above = soup.select_one(danger_above_tag).text
display(danger_above)
area_forecast = soup.select_one(area_tag).text
display(area_forecast)
date_forecast = soup.select_one(date_tag).text
display(date_forecast)
danger_above = soup.select_one(danger_above_tag).text
display(danger_above)
danger_near = soup.select_one(danger_near_tag).text
display(danger_near)
danger_below = soup.select_one(danger_below_tag).text
display(danger_below)

'Moderate'

'Stevens Pass'

'December 31, 2018'

'Moderate'

'Moderate'

'Moderate'

In [211]:
danger_above_tag = '#treeline-above > div.span4.elev-day1-column > div.danger-description > h4'
danger_near_tag ='#treeline-near > div.span4.elev-day1-column > div.danger-description > h4'
danger_below_tag = '#treeline-below > div.span4.elev-day1-column > div.danger-description > h4'
area_tag = '#main-content > h2'
date_tag = '#elevation-levels-header > div.span4.desktop.elevation-day-name > p'

result = []
soup = BeautifulSoup(stevens[0]['html'],'lxml')

area_forecast = soup.select_one(area_tag).text
date_forecast = soup.select_one(date_tag).text
danger_above = soup.select_one(danger_above_tag).text
danger_near = soup.select_one(danger_near_tag).text
danger_below = soup.select_one(danger_below_tag).text
result.append({'date': date_forecast,
               'area': area_forecast,
               'danger_above_treeline': danger_above,
               'danger_near_treeline': danger_near,
               'danger_below_treeline': danger_below,

              })

In [212]:
result

[{'date': 'December 31, 2018',
  'area': 'Stevens Pass',
  'danger_above_treeline': 'Moderate',
  'danger_near_treeline': 'Moderate',
  'danger_below_treeline': 'Moderate'}]

In [177]:
len(htmls)

1

In [125]:
for selector in selectors:
    print({selector[10:15]:soup.select(selector)[0].text})

{'above': 'Moderate'}
{'near ': 'Moderate'}
{'below': 'Moderate'}


In [128]:
stevens[0]['url']

'https://www.nwac.us/avalanche-forecast/avalanche-region-forecast/5176/cascade-west-stevens-pass/'

In [ ]:
for page in stevens[:2]:
        

In [194]:
textabove = soup.select_one('#elevation-levels-header > div.span4.desktop.elevation-day-name > p')
textabove.text

'January 22, 2019'

In [188]:
soup.select_one('#treeline-near > div.span4.elev-day1-column > div.danger-description > h4').text

'Considerable'

In [191]:
soup.select_one('#treeline-below > div.span4.elev-day1-column > div.danger-description > h4').text

'Moderate'

In [197]:
soup.select_one('#treeline-above > div.span4.elev-day1-column > div.danger-description > h4').text

'Considerable'